In [ ]:
conda install -c conda-forge earthengine-api

In [ ]:
pip install Seaborn

In [ ]:
 pip install Matplotlib

In [ ]:
 pip install xlwt

In [ ]:
 pip install openpyxl

In [20]:
import ee
import shapely
import json
import pandas as pd
import glob 
import os 
import numpy as np

In [15]:
#Define the following paths and parameters

#enter your projects name in Earth Engine
myproject='my_project'

#enter path of geojson file with station points
geojson_file_path = 'my_directory/Gas_plants_units_2024.geojson'

#Extract data for the following time interval and for the following variables
i_date = '2023-01-01'
f_date = '2023-12-31'
scale = 1113 #meters
ls = ["CH4","CO","NO2","SO2","HCHO","O3","AER_AI","AER_LH", "CLOUD"] 

In [9]:
#Source: https://github.com/antonkout/ee-project/blob/master/ee-project.ipynb

def ee_array_to_df(arr, list_of_bands):

    '''
    Parameters:
    - arr (list): The array obtained from ee.Image.getRegion on the client-side.
    - list_of_bands (list): A list of band names to be included in the DataFrame.
    '''

    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0] 
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]
    
    # Rename the columns of bands
    gases_1= ('CH4','CO','O3','SO2','NO2')
    for band in list_of_bands:
        for gas in gases_1:
            if gas in band:
                first_word = band.split('_')[0]
                if gas!="CH4":
                     df.rename(columns={band: (first_word + " (mol/m2)")}, inplace=True)
                else:
                    df.rename(columns={band: (first_word + " (ppb)")}, inplace=True)
                

    #for variable AER_LH
    for band in list_of_bands:
        if band== 'aerosol_height':
            df.rename(columns={band: (band + " (m)")}, inplace=True)

    #for variable HCHO
    for band in list_of_bands:
        if band== 'tropospheric_HCHO_column_number_density':
            df.rename(columns={band: ('HCHO' + " (mol/m2)")}, inplace=True)

            
    return df

In [10]:
#Source: https://github.com/antonkout/ee-project/blob/master/ee-project.ipynb

def ee_sentinel5p_get_data(par, i_date, f_date, scale, u_poi):
    
    '''
    Parameters:
    - par (str): The parameter of interest (e.g., 'CO', 'NO2', 'SO2').
    - i_date (str): The start date in 'YYYY-MM-DD' format.
    - f_date (str): The end date in 'YYYY-MM-DD' format.
    - scale (int): The scale for data retrieval.
    - u_poi (ee.Geometry.Point): The Earth Engine point representing the urban area of interest.
    ''' 
    

    col = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_'+par)

    if par!='HCHO' and par!='CH4' and par!='AER_AI' and par!='AER_LH'  and par!='CLOUD':
            valparameter = par + '_column_number_density'
            print(valparameter)
            par_val = col.select(valparameter).filterDate(i_date, f_date)
            # Get the data for the pixel intersecting the point in urban area.
            par_val_u_poi = par_val.getRegion(u_poi, scale).getInfo()
            par_df = ee_array_to_df(par_val_u_poi,[par +'_column_number_density']).reset_index(drop=True)
    else:
          if par=='HCHO':
                valparameter='tropospheric_HCHO_column_number_density'

          #CH4 is available in OFFLINE version only
          if par=='CH4':
                 col = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_'+par)
                 valparameter='CH4_column_volume_mixing_ratio_dry_air'

          if par=='AER_AI':
            valparameter='absorbing_aerosol_index'

          if par=='AER_LH':
            valparameter='aerosol_height'
          
          if par=='CLOUD':
            valparameter='cloud_fraction'
                  
          par_val = col.select(valparameter).filterDate(i_date, f_date)

          # Get the data for the pixel intersecting the point of station.
          par_val_u_poi = par_val.getRegion(u_poi, scale).getInfo()
          par_df = ee_array_to_df(par_val_u_poi,[valparameter]).reset_index(drop=True)
            

    return par_df

In [ ]:
ee.Authenticate()

In [12]:
# Initialize the library of Earth Engine
ee.Initialize(project=myproject) 

In [26]:
#Function that creates a csv file with measures from Sentinel-5p for each energy station
def products_excel (geojson_file_path,ls,i_date,f_date,scale):

    with open(geojson_file_path, 'r') as geojson_file:
        geojson_data = json.load(geojson_file)


    #Keep unique entries only, based on Name of plants (in case of multiple units in the same station)
    # Create an empty FeatureCollection GeoJSON structure
    unique_geojson = {
        "type": "FeatureCollection",
        "name": "Gas_plants",
        "crs": {
            "type": "name",
            "properties": {
                "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
            }
        },
        "features": []
    }

    unique_names=[]

    for feature in geojson_data['features']:
            
            Name=(feature['properties']['Plant name'])
            
            if Name not in unique_names:
                unique_geojson['features'].append(feature)
                unique_names.append(Name)


    #Extract data:
    for i in unique_geojson['features']:
            geometry = shapely.geometry.shape(i['geometry'])
            u_poi = ee.Geometry.Point(geometry.coords[0])
            
            #for the gasplant: a list in which each element represents a gas/parameter
            emmisions_ls_df = []
            for gas in ls:
                emmisions_ls_df.append(ee_sentinel5p_get_data(gas, i_date, f_date, scale, u_poi))
            

            # Convert 'datetime' to date
            for df in emmisions_ls_df:
                df['datetime'] = pd.to_datetime(df['datetime']).dt.date


            # Perform inner joins based on 'datetime' and number of gases (length of ls list):
            num=len(ls)
            if num>1:
                result = pd.merge(emmisions_ls_df[0], emmisions_ls_df[1], on='datetime', how='outer')
                for k in range (1,num-1):
                    
                    result = pd.merge(result, emmisions_ls_df[k+1], on='datetime', how='outer',suffixes=(None,k))

            else:
                result= pd.DataFrame(emmisions_ls_df[0])


            result = result.loc[:, ~result.columns.str.contains('time') | (result.columns == 'datetime')]
            result = result.reset_index(drop=True)
            
            
            #find duplicates based on datetime
            bool=result.duplicated(subset='datetime',keep=False)

            #create an empty df with the same columns of result
            duplicates= result.iloc[0:0]

            #fill the empty df with all duplicates of result
            rows=result.shape[0]
            for j in range (0,rows):
                if bool[j]==True:
                    duplicates=duplicates._append(result.iloc[j])

            #group by same datetimes and calculate average values for each date and gas
            average_per_gas = duplicates.groupby('datetime')[duplicates.columns[1:]].mean().reset_index()

            #final result: drop duplicates on initial df, add averages and sort all values based on datetimes
            final=result.drop_duplicates(subset=['datetime'], keep=False)._append(average_per_gas).sort_values('datetime').reset_index(drop=True)
            
            #Extract data in csv file
            string=i['properties']['Plant name']
            string=string+'_NRT'+ '.xlsx'
            final.to_excel(string)

In [ ]:
#Extract atmospheric data in excel form for each station
products_excel (geojson_file_path,ls,i_date,f_date,scale)

In [28]:
#Combine CH4 columns of all stations in a merged CH4 file
  
# use glob to get all the csv files stored in the same folder of current python file
path = os.getcwd() 
csv_files = glob.glob(os.path.join(path, "*.xlsx")) 


f=len(csv_files)

#list with names of excels
files=[]
result=pd.DataFrame()

# Initialize an empty list to store DataFrames
dfs = []

#for loop: combine all excels and add ID column based on Station name
for file in range (0,f,2): 
    df1 = pd.read_excel(csv_files[file],usecols=['CH4 (ppb)','datetime'])
    df2 = pd.read_excel(csv_files[file+1],usecols=['CH4 (ppb)','datetime'])
    frames = [df1, df2]


    ID1=(str(csv_files[file].split()[0].split("\\")[-1]))
    ID2=(str(csv_files[file+1].split()[0].split("\\")[-1]))
    
    # Add ID column to each DataFrame
    df1['ID'] = ID1
    df2['ID'] = ID2

    # Append the DataFrames to the list
    dfs.append(df1)
    dfs.append(df2)
       

# Concatenate all DataFrames in the list
result = pd.concat(dfs)

result = result.drop_duplicates(subset=["datetime", "CH4 (ppb)"])
result = result.groupby("datetime", as_index=False)


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


finaldata= pd.DataFrame()

for group_name,group_data in result:
    if len(group_data)>1:
        group_data = group_data.dropna(subset=['CH4 (ppb)'])
    if (group_data["CH4 (ppb)"]).isnull().all():
        group_data["ID"]=None

    finaldata = pd.concat([finaldata, group_data])



finaldata=finaldata.reset_index(drop=True)

#Calculate mean value for dates with multiple observations:

#Transform id column into string type because of "None" entries
finaldata["ID"] = finaldata["ID"].astype(str)


IDdata=finaldata.groupby("datetime")["ID"].apply(','.join)
IDdata=pd.DataFrame(IDdata)

meandata=finaldata.groupby("datetime")["CH4 (ppb)"].mean()
meandata=pd.DataFrame(meandata)

merged=pd.merge(IDdata, meandata, on='datetime', how='inner')

merged.reset_index(inplace=True)
#print(merged)

In [31]:
#add missing dates of selected year for CH4 data
dates = pd.date_range(start=i_date, periods=365)
dates=pd.DataFrame(dates)
dates.columns=['datetime']

final_CH4=pd.merge(dates, merged, on='datetime', how='outer')

#Replace "None" values with NaN for ID column
final_CH4.loc[final_CH4.ID == 'None', 'ID'] = np.nan


print(final_CH4)

      datetime                                                 ID    CH4 (ppb)
0   2023-01-01                                     Komotini,Thiva  1907.760876
1   2023-01-02                               Komotini,South,Thiva  1900.907657
2   2023-01-03                                                NaN          NaN
3   2023-01-04                          Komotini,Ptolemaida,Thiva  1896.959025
4   2023-01-05                                         Ptolemaida  1906.878909
5   2023-01-06                          Komotini,Megalopoli,Thiva  1898.978048
6   2023-01-07                               Komotini,South,Thiva  1904.655395
7   2023-01-08                                     Komotini,Thiva  1909.962158
8   2023-01-09                                                NaN          NaN
9   2023-01-10                                                NaN          NaN
10  2023-01-11                                                NaN          NaN
11  2023-01-12                                      

In [25]:
#Excel with merged CH4 data and station names that were used
final_CH4.to_excel("MergedCH4.xlsx")